# Toward better predictions of militarized inter-state disputes

### Aim

The aim in this notebook is to predict the occurrences of militarized inter-state disputes (MIDs) between states using some of the most common machine learning algorithms. MIDs are conflicts in which one or more states threaten, display, or use force against one or more other states. Thus, not only do they include instances of interstate wars, but also cases in which there was a credible threat to use militaristic means, or in which military force was actually used without culminating to war. This notebook includes the python code used to fine-tune and compare the performance of as many of the supervised-learning algorithms as time and my laptop's computing resources have allowed. For future work, I hope to re-run an expanded version of the analysis done here on the cloud. 

### Data

- The dataset includes some data on inter-state relations covering the period 1816-2009. 
- Each sample (row) represents the relationship between two nation-states of the international system in a given year. This relationship is undirected, meaning that for each pair of states in a given year, there is only one record. Thus, the variable capturing the occurrence of a MID reflects whether or not a militarized conflict has erupted between a given pair in a given year. It doesn't reflect which side of the dyad has initiated the conflict.
- As standard in the study of MIDs, the dataset only includes instances of MID initiations. Thus, the data doesn't include the later years of an ongoing conflict for the MIDs that lasted over more than single year. This is so because it's believed that the causes of MIDs' initiation are different from those of its perpetuation. 
- As also standard in the study of MIDs, the predictor variables are lagging by one year behind the output of interest. This is done in order to reduce the risk of reverse causality (e.g. a state experiencing a MID in a given year might increase its military spending as a result. Thus, the latter's value in the same year cannot be used to predict the former).  


### Algorithms 

- The following list of supervised-learning algorithms is considered: Logistic regression, naive Bayes, decision trees, random forests, gradient boosted decision trees, linear support vector classifiers, and deep neural networks. Originally, I intended to also consider two additional algorithms: Nearest neighbors and the Kernalized version of the support vector machines. But as it turned out during the initial tinkering phase, both algorithms were awfully slow to apply to the dataset at hand.
- With the exception of Gaussian naive bayes, each of the algorithms considered here has one important parameter that will be fine-tuned. Thus, and in order to minimize the risk of overfitting, the data will be split into a training and test sets. The fine-tuning processes will be carried-out on the training set using Grid Search with cross-validation. The predictive performance of the best model of each algorithm (as identified by the cross-validated Grid Search) will then be evaluated against the test set. 
- In order to keep things computationally manageable, I only attempt to fine-tune one parameter for each algorithm. The chosen parameters are the important ones suggested by Müller and Guido (2017, chapter 2)\*. Specifically for:
        - Decision tree & Random forests: maximum tree depth
        - Logistic regression and Linear SVC: regularization parameter (C)
        - Gradient boosted decision trees: learning rate
        - Deep neural networks: number and sizes of hidden layers
 

### Evaluation metrics

- I principally use the __area under the Receiver Operating Characteristic (ROC) curve__ as the evaluation metric to optimize the different algorithms.  This is a suitable metric, I believe, because the data is highly imbalanced (the output variable 'mid_leading' is equal to one in less than 0.4% of the samples). 
- In the later parts of the analysis, I consider the effects of changing the decision threshold of the best performing models. To that end, I'll also use two additional metrics: the __Youden's J statistic__ (which captures the tradeoff between TPR and FPR) and the __f1-score__ (which captures the tradeoff between precision and recall).

### On parallel GridSearch

Naturally, running the Grid Search process in parallel (using the n_jobs option) is advantageous. Unfortunately, on my laptop (which runs Windows 7 Enterprise), any value other than one for the n_jobs option produces an error. After some investigation, it turned out that the issue is Windows-related (see https://joblib.readthedocs.io/en/latest/parallel.html), and it can be fixed by restructuring the code as follows :
   
    import ....

    def function1(...):
        ...

    def function2(...):
        ...

    ...
    if __name__ == '__main__':
        # do stuff with imports and functions defined about
        ...

This restructuring makes the n_jobs option work, but it doesn't enable the interactive manipulation of the code and data in separate inputs (because each function in the above solution has to be in the same input). Thus, I have written two notebooks. In the notebook herein, I have kept the code into separate inputs just the way I did during developing the code. The second notebook (in a file called __n_jobs_fix.ipynb__) includes a virtually identical code, but one that is re-organized according to the above structure in order to make the n_jobs option of GridSearchCV work.

### To re-do the optimization
The Grid Search optimization process reads and stores the output from and into the file, _results.csv_. If you want to re-do the Grid search optimization process from scratch, please delete the results.csv file.

-----------------------------------------------------------------------------------------------------
*Müller, Andreas C. and Sarah Guido. “Supervised Learning.” Chapter 2 (pages 25-69) in Introduction to Machine Learning with Python: A Guide for Data Scientists, O'Reilly Media, Inc.




#### Notebook by Tarek Oraby (tarek.oraby@gmail.com)

In [ ]:
# reset variables
%reset -f

In [2]:
# all imports necessary for this notebook
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
random.seed(12345)
np.random.seed(12345)

In [4]:
# read data
df = pd.read_csv('Source data/conflicts_dataset.csv')

In [5]:
# check no missing values exist
if df.isnull().values.any():
    print("Warning!! Some missing values were found in the input data")

In [ ]:
print("Dataset includes %d rows and %d features"% df.shape)

In [ ]:
print("Dataset includes the following features:\n" , df.columns.values, sep='')

In [ ]:
df.head()

In [ ]:
print("Features' datatypes are:\n" ,df.get_dtype_counts(), sep='')

In [ ]:
print("The dependent variable 'mid_leading' has the following distribution:\n" ,df['mid_leading'].value_counts(), sep='')
peacful_percentage = df['mid_leading'].value_counts(normalize=True)[0] * 100
print("\nWhich means that %.2f%% of the state-pairs in the dataset are (thankfully) not in conflict with one anothers" %  peacful_percentage)

In [ ]:
# The first seven featues in the dataset (i.e., year, ccode_1, ccode_2, stateabb_1, stateabb_2, statenme_1, statenme_2) are 
# identifications that are unlikely to help in obtaining a general predictor of conflict between states. Accordingly these
# will be excluded from the analysis.

# Take the features from 'cinc_1' to 'distance' in a separate array as the predictors of the output (mid_leading)
x_array = df.loc[:, 'cinc_1':'distance'].values

In [ ]:
# Rescale features of x_array to fall between 0-1 because some algorithms (e.g. SVM) yield better performance after rescaling
scaler = MinMaxScaler()
scaler.fit(x_array)
x_array = scaler.transform(x_array)

In [ ]:
# Split data into a training and test set, the former will be used for Grid Search with cross-validation
X_train, X_test, y_train, y_test = train_test_split(x_array, df['mid_leading'], stratify=df['mid_leading'], shuffle=True,  test_size=0.25)

In [ ]:
# The following defines the search space for each algorithm
# I generally consider more values for faster algorithms (e.g. decision trees) relative to slower ones (e.g. Linear SVC).
# The exception to this is the MLPClassifier, for which I explore a bunch of values despite being relatively slow.

search_space = [['GaussianNB', {}], # GaussianNB has no important parameter to fine-tune (it has a decision threshold which will be considered later). It's included here just for reference.
                ['DecisionTreeClassifier', {'max_depth': [4, 6, 8, 12, 14, 16, 18]}],
                ['RandomForestClassifier', {'max_depth': [4, 6, 8, 12]}],                
                ['LogisticRegression', {'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
                ['LinearSVC', {'C': [0.001, 1,  100]}],
                ['GradientBoostingClassifier', {'learning_rate': [0.001, 1,  100]}],
                ['MLPClassifier', {'hidden_layer_sizes': [[100, 100],
                                                          [30, 30, 30, 30],
                                                          [60, 20, 10],
                                                          [10, 10, 10, 10, 10, 10, 10]]}]]


In [ ]:
labels = ['classifier', 'parameter_grid']
results = pd.DataFrame.from_records(search_space, columns=labels)
### add columns to hold the results
results['processed'] = False
results['best_parameter'] = None
results['test_score'] = None
results['mean_crossVal_score'] = None
results['mean_crossVal_fit_time'] = None
results['mean_crossVal_score_time'] = None
results['best_estimator_settings_all'] = None
results['evaluation_metric'] = 'roc_auc' # inclded just in case there is a need in the future to optimize on a different metric
results


# On the coding strategy of Grid Search

Given the time-consuming nature of grid search, and in order to avoid losing the search results that has been already obtained in case of an unexpected error or a computer crash, I coded the Grid search so that it saves the results directly to file and to check that file before it starts optimizing a new classifier. 

In [ ]:
# only create a results.csv file if one doesn't already exist
try:
    # start by reading the results.csv file 
    results = pd.read_csv('results.csv')
except:
    results.to_csv('results.csv', index = False)
    

In [ ]:
# This method checks the results file to see if a classifier has already been processed.
def classifier_processed(classifier_name, results):
    if (results['processed'][results['classifier'] == classifier_name].iloc[0] == True):  
        return True
    else:
        return False

In [ ]:
# This method prints the results of optimization
def print_results(classifier_string, results):
    best_param = results.loc[results['classifier'] == classifier_string, ['best_parameter']].iloc[0].values[0]
    mean_crossVal_score = results.loc[results['classifier'] == classifier_string, ['mean_crossVal_score']].iloc[0].values[0]
    test_score = results.loc[results['classifier'] == classifier_string, ['test_score']].iloc[0].values[0]
    print("Best parameter: " + str(best_param))
    print("Mean cross-validation score of best: %.4f" % mean_crossVal_score)
    print("Test score of best: %.4f" %test_score)

In [ ]:
# Given that the output 'mid_leading' is imbalanced, use stratfied folds with shuffling in the cross-val of the grid search
kfold = StratifiedKFold(n_splits = 4, shuffle = True)

In [ ]:
print("\n\n***Start of grid search optimization based on area under ROC curve***\n\n")

for item in search_space:
    classifier_string = item[0]
    parameter_grid = item[1]
    
    # check the results file if this classifier has already been processed
    if classifier_processed(classifier_string, results):
        print(classifier_string + " is already optimized, with the following results \n")
        print_results(classifier_string, results)
        print("\n\n\n")
        continue
        
    print("Working on optimizing: " + classifier_string)
    print("Using the following parameter grid: " + str(parameter_grid) + "\n")
    
    classifier = eval(classifier_string + '''()''')
    
    grid_search = GridSearchCV(classifier, parameter_grid, cv=kfold, scoring='roc_auc', n_jobs=-1, verbose=10)
    grid_search.fit(X_train, y_train)
    
    test_score = None
    predictions_of_best = None
    if hasattr(classifier, 'predict_proba'):
        predictions_of_best = grid_search.best_estimator_.predict_proba(X_test)[:, 1]
        test_score = roc_auc_score(y_test,  predictions_of_best)
    elif hasattr(classifier, 'decision_function'):
        predictions_of_best = grid_search.best_estimator_.decision_function(X_test)
        test_score = roc_auc_score(y_test,  predictions_of_best)
    else: 
        print('Cannot calculate area under ROC curve')
    
        
    # update the results datadrame & file
    results.loc[results['classifier'] == classifier_string, ['processed']] = True
    results.loc[results['classifier'] == classifier_string, ['best_parameter']] = str(grid_search.best_params_)
    results.loc[results['classifier'] == classifier_string, ['test_score']] = test_score
    results.loc[results['classifier'] == classifier_string, ['mean_crossVal_score']] = grid_search.best_score_
    results.loc[results['classifier'] == classifier_string, ['mean_crossVal_fit_time']] = grid_search.cv_results_['mean_fit_time'][grid_search.best_index_]
    results.loc[results['classifier'] == classifier_string, ['mean_crossVal_score_time']] = grid_search.cv_results_['mean_score_time'][grid_search.best_index_]
    results.loc[results['classifier'] == classifier_string, ['best_estimator_settings_all']] = str(grid_search.best_estimator_)
    
    # print results of best
    print("\n\nOptimization Results of " + classifier_string + ":")
    print_results(classifier_string, results)
    
    # save results of best to file
    results.to_csv('results.csv', index = False)
    
    # save trained best model to file
    joblib.dump(grid_search.best_estimator_, classifier_string + '.pkl') 
    
    print(80 * "-")
    

In [ ]:
print('\nBased on the area under the ROC curve, the best parameters for the optimized algorithms are as follows\n')
print(results[['classifier', 'best_parameter']])
print(80 * "-")

In [ ]:
print('\nThe mean cross-val and test scores for the optimized algorithms are as follows (sorted by mean_crossVal_score)\n')
print(results[['classifier', 'mean_crossVal_score', 'test_score' ]].sort_values(by = ['mean_crossVal_score'], ascending=False))
print(80 * "-")

In [ ]:
top_ROC_classifier = results[['classifier', 'mean_crossVal_score']].sort_values(by = ['mean_crossVal_score'], ascending=False).iloc[0][0]
top_ROC_parameter = results[['best_parameter', 'mean_crossVal_score']].sort_values(by = ['mean_crossVal_score'], ascending=False).iloc[0][0]
print('Thus, based on the training data, the ' + top_ROC_classifier + ' algorithm with the\nfollowing parameter' + str(top_ROC_parameter) + ' has the highest area under the ROC curve.')
print(80 * "-")

In [ ]:
# print ROC curves based on training data
print('ROC curves based on training data')
for item in search_space:
    classifier_string = item[0]
    trained_classifier = joblib.load(classifier_string + '.pkl')
    
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_train)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_train)
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    else: 
        print('Cannot calculate area under ROC curve')
        
    plt.plot(fpr, tpr, label=classifier_string)
    plt.xlabel("FPR")
    plt.ylabel("TPR (recall)")
    plt.legend(loc= 'lower right')
    
plt.show() 
print(80 * "-")

# Balancing TPR and FPR

Now that we know which classifiers have which ROC curves, let's focus on adjusting the decision thresholds for these classifiers. Based on the ROC curves, I think it'd be desirable to discover the threshold at which we can acheive a high TPR with an accptably low FPR. In other words, I want to find out the point on the ROC curve that would yeild the maximum value for "TPR - FPR". After googling it, the latter fomula turned out to be called __Youden's J statistic__ (https://en.wikipedia.org/wiki/Youden%27s_J_statistic). 

In [ ]:
# create three new columns in the results datafame to hold the threshold at which the maximum j-score is obtained in the
# training data, as well as the values of the j-score for the training and test data 
results['j_threshold'] = None
results['j_max_train'] = None
results['j_max_test'] = None

In [ ]:
# Optimizing decision thresholds based on Youden's J-statistic
print('\n\nOptimizing decision thresholds based on Youden\'s J-statistic:\n\n')
for item in search_space:
    classifier_string = item[0]
    trained_classifier = joblib.load(classifier_string + '.pkl')
    
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_train)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_train)
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    else: 
        print('Cannot calculate area under ROC curve')
    
    # calculate j-scores for all decision thresholds
    j_scores = tpr-fpr
    
    # Find the max j-score and its threshold
    j_ordered = sorted(zip(j_scores,thresholds))
    max_j_train = j_ordered[-1][0]
    threshold_of_max = j_ordered[-1][1]
    
    
    # Now for the test data, calculate Youden's J-Score based on the identified threshold_of_max
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_test)[:, 1]
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_test)

    predictions_of_max_test = []
    for curr_prediction in predictions_of_best:
            if curr_prediction < threshold_of_max:
                predictions_of_max_test.append(0)
            else:
                predictions_of_max_test.append(1)
    
    # Youdin's j-score is also equal to (sensitivity + specificity - 1)
    sensitivity = recall_score(y_test, predictions_of_max_test)
    specificity = recall_score(y_test, predictions_of_max_test, pos_label=0)
    max_j_test = sensitivity + specificity - 1
    
    # save to the results dataframe
    results.loc[results['classifier'] == classifier_string, ['j_max_train']] = max_j_train
    results.loc[results['classifier'] == classifier_string, ['j_max_test']] = max_j_test
    results.loc[results['classifier'] == classifier_string, ['j_threshold']] = threshold_of_max
    
    print('For the classifier ' + classifier_string)
    print('On the training data, the maximum j score is ' + str(max_j_train) + ', which was obtained using the threshold: ' + str(threshold_of_max))
    print('On the test data, and at the same threshold, the j score is ' + str(max_j_test) + ' with the following classification report:\n')
    print(classification_report(y_test,  predictions_of_max_test))
    print('and the following accuracy score ' + str(accuracy_score(y_test,  predictions_of_max_test)))
    print(80 * "-", "\n")

In [ ]:
top_j_classifier = results[['classifier', 'j_max_train']].sort_values(by = ['j_max_train'], ascending=False).iloc[0][0]
print('Thus, based on the training data, the ' + top_j_classifier + ' algorithm has the highest maximum Youden\'s J-statistic, which means that it has the highest maximum difference betweeb TPR and FPR\n')
print(results[['classifier', 'j_max_train', 'j_max_test', 'j_threshold' ]].sort_values(by = ['j_max_train'], ascending=False))
print(80 * "-")

In [ ]:
# based on training data, print ROC curves highlighting the thresholds at which the maximum Youden's J-statistic is obtained
print('ROC curves with max. Youden\'s J-statistic, based on training data')
for item in search_space:
    classifier_string = item[0]
    trained_classifier = joblib.load(classifier_string + '.pkl')
    
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_train)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_train)
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    else: 
        print('Cannot calculate area under ROC curve')
        
    threshold_of_max = results.loc[results['classifier'] == classifier_string, ['j_threshold']].iloc[0][0]
    close_max_threshold = np.argmin(np.abs(thresholds - threshold_of_max))
    
    plt.plot(fpr, tpr, label=classifier_string)
    plt.plot(fpr[close_max_threshold], tpr[close_max_threshold], 'o', markersize=10,
             label=("Youden's J max"), fillstyle="none", c='k', mew=2)
    plt.xlabel("FPR")
    plt.ylabel("TPR (recall)")
    plt.legend(loc= 'lower right')
    plt.show()
    
print(80 * "-")

# Balancing precision and recall

Another desirable property of a classifier is to have a high degree of both precision and recall. In what follows, I seek the decision threshold for each of the identified classsifiers that would yield the maximum f1-score

In [ ]:
# create three new columns in the results datafame to hold the threshold at which the maximum f1-score is obtained in the
# training data, as well as the values of the f1-score for the training and test data 
results['f1_threshold'] = None
results['f1_max_train'] = None
results['f1_max_test'] = None


In [ ]:
# Optimizing decision thresholds based on the f1-score
print('\n\nOptimizing decision thresholds based on the f1-score\n\n')
for item in search_space:
    classifier_string = item[0]
    trained_classifier = joblib.load(classifier_string + '.pkl')
    
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_train)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_train, predictions_of_best)
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_train)
        precision, recall, thresholds = precision_recall_curve(y_train, predictions_of_best)
    else: 
        print('Cannot calculate precision and recall curve')
          
    max_f1_train = 0
    threshold_of_max = None
    for r, p, t in zip(recall, precision, thresholds):
        if p + r == 0: continue
        if (2*p*r)/(p + r) > max_f1_train:
            max_f1_train = (2*p*r)/(p + r) 
            threshold_of_max = t

    
    
    # Now for the test data, calculate the f1-score based on the identified threshold_of_max
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_test)[:, 1]
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_test)

    predictions_of_max_test = []
    for curr_prediction in predictions_of_best:
            if curr_prediction < threshold_of_max:
                predictions_of_max_test.append(0)
            else:
                predictions_of_max_test.append(1)
    
    max_f1_test = f1_score(y_test,  predictions_of_max_test)
    
    # save to the results dataframe
    results.loc[results['classifier'] == classifier_string, ['f1_max_train']] = max_f1_train
    results.loc[results['classifier'] == classifier_string, ['f1_max_test']] = max_f1_test
    results.loc[results['classifier'] == classifier_string, ['f1_threshold']] = threshold_of_max
    
    print('For the classifier ' + classifier_string)
    print('On the training data, the maximum f1 score is ' + str(max_f1_train) + ', which was obtained using the threshold: ' + str(threshold_of_max))
    print('On the test data, and at the same threshold, the f1 score is ' + str(max_f1_test) + ' with the following classification report:\n')
    print(classification_report(y_test,  predictions_of_max_test))
    print('and the following accuracy score ' + str(accuracy_score(y_test,  predictions_of_max_test)))
    print(80 * "-", "\n")

In [ ]:
top_f1_classifier = results[['classifier', 'f1_max_train']].sort_values(by = ['f1_max_train'], ascending=False).iloc[0][0]
print('Thus, based on the training data, the ' + top_f1_classifier + ' algorithm has the highest maximum f1-score\n')
print(results[['classifier', 'f1_max_train', 'f1_max_test', 'f1_threshold' ]].sort_values(by = ['f1_max_train'], ascending=False))
print(80 * "-")

In [ ]:
print('Precision-recall curves with maximum f1-scoares, based on training data')
for item in search_space:
    classifier_string = item[0]
    trained_classifier = joblib.load(classifier_string + '.pkl')
    
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_train)[:, 1]
        precision, recall, thresholds = precision_recall_curve(y_train, predictions_of_best)
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_train)
        precision, recall, thresholds = precision_recall_curve(y_train, predictions_of_best)
    else: 
        print('Cannot calculate precision and recall curve')
    
    threshold_of_max = results.loc[results['classifier'] == classifier_string, ['f1_threshold']].iloc[0][0]
    close_max_threshold = np.argmin(np.abs(thresholds - threshold_of_max))
    
    plt.plot(precision, recall, label=classifier_string)
    plt.xlabel("Precision")
    plt.ylabel("recall")
    plt.plot(precision[close_max_threshold], recall[close_max_threshold], '^', markersize=10,
             label=("f1 max"), fillstyle="none", c='k', mew=2)
    plt.legend(loc= 'upper right')
    plt.show()
    
print(80 * "-", "\n")

In [ ]:
# To highlight the difference between the two evaluation metrics, print ROC curves highlighting both the thresholds at which 
# the maximum Youden's J-statistic is obtained and the threshold at which the maximum f1-score is obtained
print('ROC curves with max. Youden\'s J-statistic & max. f1-score, based on training data')
for item in search_space:
    classifier_string = item[0]
    trained_classifier = joblib.load(classifier_string + '.pkl')
    
    if hasattr(trained_classifier, 'predict_proba'):
        predictions_of_best = trained_classifier.predict_proba(X_train)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    elif hasattr(trained_classifier, 'decision_function'):
        predictions_of_best = trained_classifier.decision_function(X_train)
        fpr, tpr, thresholds = roc_curve(y_train, predictions_of_best)
    else: 
        print('Cannot calculate area under ROC curve')
        
    j_threshold_of_max = results.loc[results['classifier'] == classifier_string, ['j_threshold']].iloc[0][0]
    j_close_max_threshold = np.argmin(np.abs(thresholds - j_threshold_of_max))
    
    f1_threshold_of_max = results.loc[results['classifier'] == classifier_string, ['f1_threshold']].iloc[0][0]
    f1_close_max_threshold = np.argmin(np.abs(thresholds - f1_threshold_of_max))
    
    plt.plot(fpr, tpr, label=classifier_string)
    plt.plot(fpr[j_close_max_threshold], tpr[j_close_max_threshold], 'o', markersize=10,
             label=("Youden's J max"), fillstyle="none", c='k', mew=2)
    plt.plot(fpr[f1_close_max_threshold], tpr[f1_close_max_threshold], '^', markersize=10,
             label=("f1 max"), fillstyle="none", c='k', mew=2)
    plt.xlabel("FPR")
    plt.ylabel("TPR (recall)")
    plt.legend(loc= 'lower right')
    plt.show()
    
print(80 * "-")

# Some substantive predictions

In [ ]:
# Examine the predicted probabilities of MIDs based on the classifier with the largest area under the ROC curve

roc_trained_classifier = joblib.load(top_ROC_classifier + '.pkl')
if hasattr(trained_classifier, 'predict_proba'):
    df['predictions_ROC_best'] = roc_trained_classifier.predict_proba(x_array)[:, 1]
elif hasattr(trained_classifier, 'decision_function'):
    df['predictions_ROC_best'] = roc_trained_classifier.decision_function(x_array)

In [ ]:
# for the last year in the dataset (2009)
print(df[['year', 'stateabb_1', 'stateabb_2', 'predictions_ROC_best' ]].sort_values(by = ['predictions_ROC_best'], ascending=False).query('year==2009'))

print(80 * "-")

In [ ]:
print('Mean predictions of MID by year')
plt.plot(df.groupby(['year'])['predictions_ROC_best'].mean())
plt.show()

In [ ]:
print('Mean predictions of MID by year adjusting for the average capabilities of the two states in the dyad')
plt.plot(df.groupby(['year'])['predictions_ROC_best'].mean() / (df.groupby(['year'])['cinc_1'].mean() + df.groupby(['year'])['cinc_2'].mean()))
plt.show()
print(80 * "-")

In [ ]:
# save results of best to file
results.to_csv('results.csv', index = False)

In [ ]:
results